In [ ]:
import ROOT as r
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends import backend_pdf
import os

In [ ]:
import sqlite3

def readSqlitedb(database="/cvmfs/icarus.opensciencegrid.org/products/icarus/icarus_data/v09_62_00/icarus_data/database/ChannelMapICARUS.db", table="pmt_placements"):

    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(database)
    df = pd.read_sql_query("SELECT * from {}".format(table), con)

    con.close()

    return df

def get_channel_id(db, board, digitizer_channel):
    
    ch = db.loc[(db['digitizer_label']==board) & (db['digitizer_ch_number']==digitizer_channel), ['channel_id']]
    # print(board, ",", digitizer_channel, "-->", ch.values[0][0])

    return ch.values[0][0]

def get_PMT_id(db, channels):
     
    if np.isscalar(channels):
        pmt_id = db[db.channel_id==channels].pmt_id.values[0]
        return pmt_id
    else:
        pmt_ids = [ db[db.channel_id==ch].pmt_id.values[0] for ch in channels ] 
        return pmt_ids

def PMTid_to_channel(pmt_ids):
    
    geo = readSqlitedb()
    
    if np.isscalar(pmt_ids):
        channel = geo[geo.pmt_id==pmt_ids].channel_id.values[0]
        return channel
    else:
        channels = [ geo[geo.pmt_id==pmt].channel_id.values[0] for pmt in pmt_ids ] 
        return channels

geodb = readSqlitedb()

In [ ]:
def get_bins_edges(bin_centers):

    bin_edges = []
    bin_edges.append(0.0) #first bin left edge

    for i in range(len(bin_centers)):
        length = 0
        if i == (len(bin_centers)-1):
            length = bin_centers[i]-bin_centers[i-1]
        else:
            length = bin_centers[i+1]-bin_centers[i]
        bin_edges.append(bin_centers[i]+length/2.)

    #print(bin_edges)
    return bin_edges

In [ ]:
from scipy.optimize import curve_fit

def poissonGauss(x, amplitude, n, mu, q, sigma ):
    return amplitude*(np.power(mu,n)*np.exp(-1.0*mu)/np.math.factorial(n)*np.exp(-1.0*(x-q*n)*(x-q*n)/(2.0*n*sigma*sigma))/(sigma*np.sqrt(2.0*np.pi*n)))

def IdealSER(x, mu, q, sigma, amplitude):  
    val = 0
    for n in range(1,5): #da 1 a 4
        val += poissonGauss(x, amplitude, n, mu, q, sigma)
    return val

In [ ]:
def plot_single_PE_fit_pdf(filedir, PMT, geodb, pdf, max, min):
    
    channel = PMTid_to_channel(PMT)
    print("PMT ID "+str(PMT)+" - Channel ID "+str(channel))
    
    hname = "hintegral"+str(channel)
    hcharge = dir.Get(hname)
    
    binned_events = []
    bin_centers = []
    for i in range(1,hcharge.GetNbinsX()+1):
        binned_events.append(hcharge.GetBinContent(i))
        bin_centers.append(hcharge.GetXaxis().GetBinCenter(i))
    bin_edges = get_bins_edges(bin_centers)
    
    fbin_centers = [x for x in bin_centers if x < max]
    fbin_edges = [x for x in bin_edges if x < max]
    length = len(fbin_centers)
    fbinned_events = binned_events[:length]

    imin = 0
    for i in range(0, length):
        if fbin_centers[i] < min:
            imin = i
        
    xs = fbin_centers[imin+1:]
    ys = fbinned_events[imin+1:]

    param = [ 0.1, 0.45, 0.3, np.max(ys)] # mu, q, sigma, amplitude
    param, pcov = curve_fit(IdealSER, xs, ys, p0=param)#, bounds=(bounds[:,0],bounds[:,1]) )
    param_errors = np.diag(pcov)**0.5
    ys_fitted = IdealSER(xs, *param)

    fig = plt.figure()
    plt.hist(fbin_centers, bins=fbin_edges, weights=fbinned_events, histtype="step", label="Run "+str(run)+" - PMT ID "+str(PMT), lw=1.5)

    lab= "SER fit: \n"
    lab+="$\mu$: %.2f $\pm$ %2.e\n" % ( param[0], param_errors[0] ) 
    lab+="q: %.3f $\pm$ %3.e\n" % ( param[1], param_errors[1] )
    lab+="$\sigma$: %.3f $\pm$ %3.e\n" % ( param[2], param_errors[2] )
    lab+="A: %.0f $\pm$ %.0f\n" % ( param[3], param_errors[3] )
    plt.plot(xs, ys_fitted,'-', lw=2.0, color='red', label=lab)

    plt.grid(color='0.95')
    plt.xlabel(r'Pulse charge [$10^7$ electrons]')
    plt.ylabel("# events")
    plt.title("Single photoelectron charge distribution")
    plt.margins(x=0.00)
    plt.legend()
    
    pdf.savefig( fig )
    plt.close()
    return pdf

def plot_single_PE_fit(filedir, PMT, geodb, max, min):
    
    channel = PMTid_to_channel(PMT)
    print("PMT ID "+str(PMT)+" - Channel ID "+str(channel))
    
    hname = "hintegral"+str(channel)
    hcharge = dir.Get(hname)
    
    binned_events = []
    bin_centers = []
    for i in range(1,hcharge.GetNbinsX()+1):
        binned_events.append(hcharge.GetBinContent(i))
        bin_centers.append(hcharge.GetXaxis().GetBinCenter(i))
    bin_edges = get_bins_edges(bin_centers)
    
    fbin_centers = [x for x in bin_centers if x < max]
    fbin_edges = [x for x in bin_edges if x < max]
    length = len(fbin_centers)
    fbinned_events = binned_events[:length]

    imin = 0
    for i in range(0, length):
        if fbin_centers[i] < min:
            imin = i
        
    xs = fbin_centers[imin+1:]
    ys = fbinned_events[imin+1:]

    param = [ 0.1, 0.45, 0.3, np.max(ys)] # mu, q, sigma, amplitude
    param, pcov = curve_fit(IdealSER, xs, ys, p0=param)#, bounds=(bounds[:,0],bounds[:,1]) )
    param_errors = np.diag(pcov)**0.5
    ys_fitted = IdealSER(xs, *param)

    fig = plt.figure()
    plt.hist(fbin_centers, bins=fbin_edges, weights=fbinned_events, histtype="step", label="PMT ID "+str(PMT), lw=1.5)

    lab= "SER fit: \n"
    lab+="$\mu$: %.2f $\pm$ %2.e\n" % ( param[0], param_errors[0] ) 
    lab+="q: %.3f $\pm$ %3.e\n" % ( param[1], param_errors[1] )
    lab+="$\sigma$: %.3f $\pm$ %3.e\n" % ( param[2], param_errors[2] )
    lab+="A: %.0f $\pm$ %.0f\n" % ( param[3], param_errors[3] )
    plt.plot(xs, ys_fitted,'-', lw=2.0, color='red', label=lab)

    plt.grid(color='0.95')
    plt.xlabel(r'Pulse charge [$10^7$ electrons]')
    plt.ylabel("# events")
    plt.title("Single photoelectron charge distribution")
    plt.margins(x=0.00)
    plt.legend()
    
    return fig

In [ ]:
run = 9627
filepath = "/icarus/data/users/mvicenzi/pmt-calibration/histograms/pulseDistributionHist_run"+str(run)+".root"
figspath = "/icarus/data/users/mvicenzi/pmt-calibration/figs/"
file = r.TFile(filepath,"READ")

In [ ]:
channel = 93
hname = "hintegral"+str(channel)
dir = file.Get("bkgcalibration")
hcharge = dir.Get(hname)
#hcharge.GetXaxis().SetRangeUser(0.,3.)
#c = r.TCanvas("canvas"+str(channel),"The Canvas Title",800,600)
#hcharge.Draw()
#c.Draw()

In [ ]:
fig = plot_single_PE_fit(dir, channel, geodb, max=3.01, min=0.3)
fig.savefig(figspath+"singlePE_charge_run"+str(run)+"_channel"+str(channel)+".png")
fig.savefig(figspath+"singlePE_charge_run"+str(run)+"_channel"+str(channel)+".pdf")

In [ ]:
offPMTs = [1, 111, 143, 166, 192, 230, 238, 254, 222, 302, 309, 340, 353, 290 ]
ma = 3.01
mi = 0.3

pdf = backend_pdf.PdfPages(figspath+"pulse_charge_fits_run"+str(run)+"_min"+str(mi)+"_max"+str(ma)+".pdf")
for PMT in range(1,361):
    
    if PMT in offPMTs:
        continue
    
    try:
        pdf = plot_single_PE_fit_pdf(dir, PMT, geodb, pdf, max=ma, min=mi)
    except:
        continue

print("ALL DONE")
pdf.close()